# Extrahierung der Textstufen

In [2]:
import lxml
from lxml import etree
import pandas as pd

In [3]:
tree = etree.parse("C:/Users/jO/Desktop/DH_MA_Arbeit/coding/modelliert_output_pos_vwdazb_trf.xml")

In [25]:
ns = {"t":'http://www.tei-c.org/ns/1.0'}
xml = "{http://www.w3.org/XML/1998/namespace}"

In [8]:
poem = tree.xpath("//t:div[@type='poem']/descendant-or-self::text()", namespaces=ns)

In [9]:
witnesses = tree.xpath("//t:div[@type='witness']/descendant-or-self::text()", namespaces=ns)

In [15]:
# amount of poems in xml
amountpoems = len(tree.xpath("//t:div[@type='poem']", namespaces=ns))
print(amountpoems)

1


In [17]:
# amount of witnesses in xml
amountwitnesses = len(tree.xpath("//t:div[@type='witness']", namespaces=ns))
print(amountwitnesses)

7


In [18]:
# amount of published in xml
amountpublished = len(tree.xpath("//t:div[@type='published']", namespaces=ns))
print(amountpublished)

1


In [21]:
# alle xml:ids der witnesses
tree.xpath("//t:div[@type='witness']/@xml:id", namespaces=ns)

['nr.1', 'nr.2', 'nr.3', 'nr.4', 'nr.5', 'nr.6', 'nr.af']

In [27]:
witness1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']/descendant-or-self::*/text()", namespaces=ns)
print(witness1)

['\n                    ', '\n                        ', 'der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV \n                            schließen_/VERB\n                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT \n                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN \n                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN\n                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN \n                            in_/ADP der_/DET ', 'Brust_/NOUN ,_/PUNCT \n                            ', 'auf_/ADP die_/DET Finger_/NOUN', ' mit_/ADP der_/DET linken_/ADJ Hand_/NOUN \n                            3_/NUM mal_/ADV geteilt_/VERB', ' \n                            ', 'Westentasche_/NOUN ,_/PUNCT', ' ', '(_/PUNCT', ' ', 'oder_/CCONJ als_/SCONJ \n                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX \n                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ \n                 

### Extrahierung vom "originalen" Typoskript: 
- <add ignorieren
- <reg ignorieren
- <del einbeziehen
- <orig einbeziehen

In [53]:
wit1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']", namespaces=ns)
linegroups = tree.xpath("//t:lg", namespaces=ns)
lines = tree.xpath("//t:l", namespaces=ns)

for linegroups in wit1: 
    for lines in linegroups: 
        wit1typ = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']//t:l/text() | //t:div[@type='witness' and @xml:id='nr.1']//t:del/text() | //t:div[@type='witness' and @xml:id='nr.1']//t:orig/text()", namespaces=ns)
        wit1typ = "".join(wit1typ)
        wit1typ.replace("\n", "").replace("\r", "")
print("Originaltyposkript:", wit1typ)

Originaltyposkript: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT die_/DET ihre_/DET 
                            Gedanken_/NOUN durch_/ADP die_/DET Feder_/NOUN 
                            sichtbar_/ADV zu_/PART mac

### Extrahierung von @change-Values Versionen: 
- <add  von @change="_pen" <hi>einbeziehen</hi>
- <reg von @change="_pen" einbeziehen
- <del einbeziehen, wenn es nicht @change="_pen" ist 
- <orig einbeziehen, wenn <reg nicht von @change="_pen" ist

In [5]:
# alle Versionen nach den verschiedenen @change-Values:
# mal nur aus dem ersten Textträger <div type='witness' @xml:id='nr.1'>

divwit1 = tree.xpath("//t:div[@type='witness' and @xml:id='nr.1']//t:l/descendant-or-self::*", namespaces=ns)
allattributevalues = set()

for child in divwit1:
    attribute_value = child.attrib.get('change')
    if attribute_value:
        if attribute_value not in allattributevalues:
            allattributevalues.add(attribute_value)
print(f"Unterschiedliche Revisionen: {allattributevalues}\n")
            
        
for attributevalue in allattributevalues:
    textname = f"{attributevalue}text"
    text = tree.xpath(f"""//t:div[@type='witness' and @xml:id='nr.1']//t:l/text() | 
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:add[@change='{attributevalue}']/text() | 
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:del[not(@change='{attributevalue}')]/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:reg[@change='{attributevalue}']/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
    //t:div[@type='witness' and @xml:id='nr.1']//t:l//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)

    textname = "".join(text)
    textname.replace("\n", "").replace("\r", "")
    print(f"Typoskript mit Revisionen nach {attributevalue}:\n {textname} \n \n")


Unterschiedliche Revisionen: {'#typewriter', '#black_pen'}

Typoskript mit Revisionen nach #typewriter:
 der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT  ihre_/DET 
                            Gedanken_/NOUN durch_/ADP

### Versuch: Extrahierung aller Textstufen nach vorkommenden @change Attributevalues aller Textzeugen

In [50]:
# Annäherung an Extrahierung von allem (Original und Textstufen nach @change-values) mit Änderung des xml-Files
poem = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div", namespaces=ns)
div = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div", namespaces=ns)
#divline = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']//t:l/descendant-or-self::*", namespaces=ns)

processed_divs = set()  # um die durchgegangenen divs zu speichern

def addtextasdiv(text, divname): 
    if divname in processed_divs:
        return
    else:
        text = "".join(text)
        text.replace("\n", "").replace("\r", "")
        print("Originaltyposkript:", text)
        # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen:
        newtag = etree.Element("div")
        # für dieses div ein @type-Attribute mit dem Value bestehend aus: Name des Ursprungs-div und "original"
        newtag.set("type", f"{divname}_original")
        # den extrahierten text als Inhalt des neuen divs definieren
        newtag.text = text
        # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen 
        div.append(newtag)
        #ende original herausnehmen
        processed_divs.add(divname) 

for div in poem: 

    divtype = div.attrib.get('type')
    divid = div.attrib.get(f'{xml}id')
    divname = f"{divtype}_{divid}"
    print(divname) 
    
#original herausnehmen: 
    if divtype == "witness": 
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
        addtextasdiv(text, divname)

    

    #allattributevalues = set()


    tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)


witness_nr.1
Originaltyposkript: der_/DET Körper_/NOUN will_/AUX sich_/PRON ja_/ADV 
                            schließen_/VERB
                            ,_/PUNCT rufst_/VERB du_/PRON ,_/PUNCT 
                            Verdruss_/NOUN und_/CCONJ Krankheit_/NOUN 
                            als_/ADP Göttin_/NOUN aus_/ADP der_/DET Maschine_/NOUN
                            (_/PUNCT die_/DET Zigeunerspielkarten_/NOUN 
                            in_/ADP der_/DET  
                            Westentasche_/NOUN ,_/PUNCT  oder_/CCONJ als_/SCONJ 
                            müssten_/AUX wir_/PRON erinnert_/VERB werden_/AUX 
                            dass_/SCONJ wir_/PRON nicht_/PART Götter_/NOUN sondern_/CCONJ 
                            Menschen_/NOUN sind_/AUX ,_/PUNCT die_/PRON es_/PRON
                            nötig_/ADV haben_/VERB ,_/PUNCT die_/DET ihre_/DET 
                            Gedanken_/NOUN durch_/ADP die_/DET Feder_/NOUN 
                            sichtbar_/ADV

In [ ]:
# alle Versionen nach den verschiedenen @change-Values:
# für das gesamte xml-Dokument:

poem = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/descendant-or-self::*", namespaces=ns)
div = tree.xpath("//t:div[@type='poem' and @xml:id='vielleichtweildasAugezuerstbricht']/t:div[@type='witness']//t:l/descendant-or-self::*", namespaces=ns)

for div in poem: 

    divtype = div.attrib.get('type')
    divid = div.attrib.get('xml:id')
    divname = f"{divtype}_{divid}"
                         
    allattributevalues = set()

    #original herausnehmen:   
    for child in div: 
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']/t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
       
        text = "".join(text)
        text.replace("\n", "").replace("\r", "")
        print("Originaltyposkript:", text)
        
        # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen:
        newtag = etree.Element("div")
        # für dieses div ein @type-Attribute mit dem Value bestehend aus: Name des Ursprungs-div und "original"
        newtag.set("type", f"{divname}_original")
        # den extrahierten text als Inhalt des neuen divs definieren
        newtag.text = text
        # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen 
        div.append(newtag)
        #ende original herausnehmen

    # alle so viele Versionen extrahieren, wie es unterschiedliche change-Attribute-Values gibt (#typewriter, #blue_pen, #black_pen etc.) 
    for child in div:
        attribute_value = child.attrib.get('change')
        if attribute_value:
            if attribute_value not in allattributevalues:
                allattributevalues.add(attribute_value)
    print(f"Unterschiedliche Revisionen: {divname} {allattributevalues}\n")
                
            
    for attributevalue in allattributevalues:
        textname = f"{attributevalue}text"
        text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
        //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", namespaces=ns)
    
        text = "".join(text)
        text.replace("\n", "").replace("\r", "")
        print(f"Typoskript mit Revisionen nach {attributevalue}:\n {text} \n \n")

        # neues Tag für den extrahierten Text mit dem Tagnamen "div" erstellen:
        newtag = etree.Element("div")
        # für dieses div ein @type-Attribute mit dem Value bestehend aus: Name des Ursprungs-div und des zugehörigen @change-Values
        newtag.set("type", f"{divname}_{attributevalue}")
        # den extrahierten text als Inhalt des neuen divs definieren
        newtag.text = text
        # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen 
        div.append(newtag)
        

tree.write("extrahiert_vwdazb.xml", encoding="utf-8", xml_declaration=True)